In [1]:
import sys
!{sys.executable} -m pip install --no-cache-dir torch
!{sys.executable} -m pip install --no-cache-dir torchvision

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 7.5 MB/s eta 0:00:000:00:01m eta 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 6.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 6.0 MB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 6.1 MB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 8.9 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.3 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 8.6 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━

In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from timeit import default_timer as timer
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms

# Writer will output to ./runs/ directory by default
writer = SummaryWriter()

ModuleNotFoundError: No module named 'torch'

In [92]:
batch_size = 64
num_classes = 3
learnin_rate = 0.0005
num_epochs = 10
input_size=np.asarray((128,64,1))
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [93]:
print(torch.cuda.is_available())

print(torch.__version__)

True
2.1.2+cu121


In [2]:
dataset = torch.load('images_perso.pth')
labels = torch.load('labels_perso.pth')
def index_vers_label(i):
    liste = sorted(os.listdir("./jeu_de_donnee_maison"))
    return liste[i]
    

NameError: name 'torch' is not defined

In [95]:
class handDataset(torch.utils.data.Dataset):

    def __init__(self):
        self.x = dataset
        self.y = labels
        self.n_samples = len(dataset)

    def __getitem__(self, index):
        return self.x[index], self.y[index]

    def __len__(self):
        return self.n_samples

In [96]:
generator = torch.Generator().manual_seed(42)

train_size = int(len(dataset) * .7)
test_size = len(dataset) - train_size

hand_dataset = handDataset()
train_dataset, test_dataset = torch.utils.data.random_split(hand_dataset, [train_size, test_size], generator=generator)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [1]:
def get_pretrained_models(model_name = 'resnet', num_classes = 2, freeze_prior = True, use_pretrained=True):
    
    from torchvision import models
    from utils import set_parameter_requires_grad
    
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, freeze_prior)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, freeze_prior)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, freeze_prior)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, freeze_prior)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, freeze_prior)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, freeze_prior)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()
        
    if True:
        from torchsummary import summary
        summary(model_ft,(3,input_size,input_size))
        print(model_ft)

    return model_ft, input_size

In [2]:

class CNN_model(nn.Module):

    def __init__(self, num_classes):
        super(CNN_model, self).__init__()
        self.conv_layer1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5)
        size=input_size-5+1
        self.max_pool1 = nn.MaxPool2d(kernel_size=2)
        size=size/2
        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        size=size-3+1
        self.max_pool2 = nn.MaxPool2d(kernel_size=2)
        size=size/2
        print(size)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(int(size[0])*int(size[1])*64,128)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(128, num_classes)

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.max_pool1(out)
        out = self.conv_layer2(out)
        out = self.max_pool2(out)
        out = self.flatten(out)
        #896x14 and 12544x128)
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)

        return out


NameError: name 'nn' is not defined

In [99]:
cnn_model = CNN_model(num_classes).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(cnn_model.parameters(), lr=learnin_rate, weight_decay=0.005, momentum=0.9)

total_step = len(train_loader)

[30.   14.   -1.75]


In [100]:
for epoch in range(num_epochs):

    for i, (images, labels) in tqdm(enumerate(train_loader)):
        images=images.to(device)
        labels=labels.to(device)
        
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

6it [00:00, 18.81it/s]


Epoch [1/10], Loss: 1.6153


6it [00:00, 83.40it/s]


Epoch [2/10], Loss: 0.8001


6it [00:00, 86.29it/s]


...

Epoch [8/10], Loss: 0.0517


6it [00:00, 92.30it/s]


Epoch [9/10], Loss: 0.0317


6it [00:00, 91.95it/s]

Epoch [10/10], Loss: 0.0237


In [2]:
with torch.no_grad():
    correct = 0
    total = 0
    start = timer()
    for images, labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        labels_class=torch.max(labels,1)
        #print(labels_class)
        for i,sample in enumerate(labels_class[1]):
            #print(sample)
            #print(predicted[i])
            if sample==predicted[i]:
                correct += 1
    end=timer()
    
    print('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))
    print("TIme of inference =" + str((end-start)/total) +"s")



NameError: name 'torch' is not defined

In [1]:
from PIL import Image
tr = transforms.Compose([
        transforms.Resize((128, 64)),
        transforms.ToTensor(),
        #transforms.RandomHorizontalFlip(),  
        #transforms.RandomRotation(90),  
        #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  
        transforms.Normalize(mean=[0], std=[0.5])  # Normalizing mean and std values
    ])

correct = 0
total = 0
for i in ["feuille", "ciseau", "pierre"]:
    for j in os.listdir("./validation/"+i):
        total+=1
        image = Image.open("./validation/"+i+"/"+j).convert('L')
        image_tensor = tr(image)
        image_tensor=image_tensor.to(device)
        image_batch = image_tensor.unsqueeze(0)
        predicted=cnn_model(image_batch).cpu().data.numpy()
        if index_vers_label(int(max(predicted[0]))))==i:
            correct+=1
print(correct/total*100)


NameError: name 'transforms' is not defined

## affichage des résultats

In [103]:
%load_ext tensorboard
%tensorboard --logdir=runs

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 4812), started 2:59:57 ago. (Use '!kill 4812' to kill it.)

In [104]:
import cv2 as cv
import numpy as np
from PIL import Image


img_path = "./WIN_20240111_10_48_03_Pro (2).jpg"
img = cv.imread(img_path)
fgbg = cv.createBackgroundSubtractorMOG2()
fgmask = fgbg.apply(img)
array_im=cv.cvtColor(fgmask, cv.COLOR_BGR2RGB)
pil_image=Image.fromarray(array_im)
pil_image.show()
hsvim = cv.cvtColor(img, cv.COLOR_BGR2HSV) 
lower = np.array([0, 48, 80], dtype = "uint8") 
upper = np.array([50, 255, 255], dtype = "uint8") 
skinRegionHSV = cv.inRange (hsvim, lower, upper) 
blurred= cv.blur (skinRegionHSV, (2,2))
ret, thresh = cv.threshold (blurred, 0,255, cv. THRESH_BINARY) 
array_im=cv.cvtColor(thresh, cv.COLOR_BGR2RGB)
pil_image=Image.fromarray(array_im)
pil_image.show()



## Sauvegarde du modèle

In [ ]:
model_scripted = torch.jit.script(cnn_model) # Export to TorchScript
model_scripted.save('sauvegarde.pt') # Save